# Toronto Clustering

#### Importing needed libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geocoder --yes
import geocoder

print('Libraries imported.')

Libraries imported.


## 1. Download the file and get it into the wanted Dataframe form:

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

In [3]:
print(df.shape)
df.head(10)

(290, 3)


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


#### Drop cells with a borough that is Not assigned:

In [4]:
df = df[df[1]!='Not assigned']
df.head(15)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


Now the shape of the Dataframe is:

In [5]:
df.shape

(213, 3)

So 77 raws deleted.

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. (see line 9):

In [6]:
df.iloc[7,2]=df.iloc[7,1] #(index=7)
df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


#### Reseting index:

In [7]:
df.reset_index(inplace=True)
df.head()

,index,0,1,2
0,0,Postcode,Borough,Neighbourhood
1,3,M3A,North York,Parkwoods
2,4,M4A,North York,Victoria Village
3,5,M5A,Downtown Toronto,Harbourfront
4,6,M5A,Downtown Toronto,Regent Park


In [8]:
df.drop('index',axis=1, inplace=True)
df.head(6)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights


#### More than one neighborhood can exist in one postal code area:

In [10]:
df1 = df[df[0].duplicated()==True].reset_index() #duplicate dataframe
df1.head()

,index,0,1,2
0,4,M5A,Downtown Toronto,Regent Park
1,6,M6A,North York,Lawrence Manor
2,10,M1B,Scarborough,Malvern
3,13,M4B,East York,Parkview Hill
4,15,M5B,Downtown Toronto,Garden District


In [11]:
s=0
for i in df1.iloc[:,0].tolist():
    for j in range(i):
        if df.iloc[j,0]==df1.iloc[s,1]:
            df.iloc[j,2]=df.iloc[j,2]+','+df1.iloc[s,3]
    s=s+1

In [12]:
df.drop(df1['index'].tolist(),axis=0,inplace=True)
df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Queen's Park,Queen's Park
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"
11,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens,Parkview Hill"


In [13]:
df.shape

(104, 3)

#### Get it in the final form:

In [14]:
df.rename(columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'},inplace=True) #rename columns
df.drop(index=0,inplace=True) #drop column raw
df.reset_index(inplace=True) #reset index
df.drop('index',axis=1,inplace=True) #drop index column
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [18]:
print(df.shape)
df.tail()

(103, 3)


,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


#### Let's find the latitude and lonitude of postal codes:

We can do that using geocoder but after the changes of the instructor we have a new excel file.

#### 1st Way:

In [37]:
#list_lat=[]
#list_lng=[]

# initialize your variable to None
#lat_lng_coords = None

#for i in range(103):
#    postal_code=df.iloc[i,0]
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#    list_lat.append(lat_lng_coords[0])
#    list_lng.append(lat_lng_coords[1])
#    lat_lng_coords = None

Add lists to the Dataframe and take the final form:

In [80]:
#df_lat=pd.DataFrame(list_lat,columns={'Latitude'})
#df_lng=pd.DataFrame(list_lng,columns={'Longitude'})

In [222]:
#neighborhoods = pd.concat([df, df_lat,df_lng], axis=1)

In [228]:
#print(neighborhoods.shape)
#neighborhoods.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


#### 2nd Way:

In [43]:
df_lat_lng = pd.read_csv("C:/Users/Sotiris/Desktop/python works/applied data science specialization/Applied data science cupstone/Geospatial_Coordinates (1).csv", sep=',')
print(df_lat_lng.shape)
df_lat_lng.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
column_names=['Latitude', 'Longitude']
df_ll=pd.DataFrame(columns=column_names)
df_ll

,Latitude,Longitude


In [45]:
for i in range(103):
    for j in range(103):
        if df.iloc[i,0]==df_lat_lng.iloc[j,0]:
            df_ll=df_ll.append({'Latitude':df_lat_lng.iloc[j,1],
                              'Longitude':df_lat_lng.iloc[j,2]}, ignore_index=True)
            

In [57]:
neighborhoods=pd.concat([df, df_ll], axis=1)

In [62]:
print(neighborhoods.shape)
neighborhoods.head(10)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


And we take a DataFrame same as in 1st Way, as we should.